In [14]:
# Код серверной части на Pyton с фреймворком Flask
# Подключение необходимых библиотек и модулей
from flask import Flask, request, jsonify, session, make_response
from flask_cors import CORS
from flask_mysqldb import MySQL
from flask_bcrypt import Bcrypt
from PIL import Image
from scipy.interpolate import splprep, splev
from werkzeug.utils import secure_filename
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
import random
import os
import cv2
import numpy as np
import time
import uuid
from flask import Flask, request, jsonify, make_response, session

# Создание экземпляра приложения Flask
app = Flask(__name__)

# Конфигурация для подключения к базе данных MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_PORT'] = 3306
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'root'
app.config['MYSQL_DB'] = 'diplom'

# Конфигурация сессионных cookies
app.config['SESSION_COOKIE_HTTPONLY'] = False  # Ограничение доступа к сессионному cookie через JavaScript
app.config['SESSION_COOKIE_SAMESITE'] = 'Lax'  # Ограничение отправки сессионного cookie только при запросах с того же сайта
app.config['SESSION_COOKIE_PATH'] = '/'        # Путь, на котором доступно сессионное cookie
app.config['SESSION_COOKIE_SECURE'] = False    # Передача сессионного cookie только через HTTPS

# Конфигурация загрузки файлов
app.config['ALLOWED_EXTENSIONS'] = {'png'}
app.config['UPLOAD_FOLDER'] = 'C:\server\data\htdocs\Diplom\my-app\public\images'

# Инициализация расширений
# Разрешение кросс-доменных запросов
cors = CORS(app, resources={r"/api/*": {"origins": "*"}}, allow_methods=["GET", "POST"])
# Настройка секретного ключа для подписи JWT токенов
app.config['JWT_SECRET_KEY'] = 'dvrgthdtjdhncyhsth'  
# Инициализация JWTManager
jwt = JWTManager(app)
# Инициализация MySQL
mysql = MySQL(app)
# Для работы с хэшированием паролей
bcrypt = Bcrypt(app)
# Инициализация MySQL
app.secret_key = os.urandom(30)
# Переменная для хранения сгенерированного пароля
codekey = None

# Функция генерации одноразового кода
def generate_kod(filename):
    num_pixels = 4 # количество точек ломаной линии
    image = Image.open(filename)
    width, height = image.size
    pixels = np.array(image)
    # Цикл для получения 4 случайных пикселей определенного цвета      
    while True:  
        # получение координат случайного пикселя
        random_pixel = (random.randint(0, height - 1), random.randint(0, width - 1)) 
        # определнение цвета случайного пикселя
        target_color = pixels[random_pixel] 
        # получение 4 случайных пикселей заданного цвета
        mask = np.all(pixels == target_color, axis=-1)
        colored_pixel_indices = np.column_stack(np.where(mask))
        if len(colored_pixel_indices) >= num_pixels:
            break  
    # аппроксимация сплайном 3 порядка 
    # для получения начального заполнения генератора
    selected_pixels = colored_pixel_indices[random.sample(range(len(colored_pixel_indices)), num_pixels)]
    x, y = selected_pixels[:, 1], selected_pixels[:, 0]
    tck, u = splprep([x, y], k=3, s=0)
    coeff_y = np.array(tck[1])
    # получение случайного кода с помощью генератора
    odnomer_array = (abs(coeff_y.astype(int))).flatten()
    zatravka = ''.join(map(str, odnomer_array))
    code = str(random.randint(0, 999999)).zfill(6)
    return code
  
# Функция проверки расширения файла    
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

# Функция поиска лица на изображении 
def face(image_path):
    # Загрузка предварительно обученной модели для обнаружения лиц
    cascade_path = 'C:/Users/sumarn/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path) 
    image = cv2.imread(image_path)   
    # Преобразование фото в черно-белое
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Обнаружение лиц
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     # Вывод результатов 
#     for (x, y, w, h) in faces:
#         cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
#     # Отображение фото с обнаруженными лицами
#     cv2.namedWindow('Faces Detected', cv2.WINDOW_NORMAL)
#     cv2.imshow('Faces Detected', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    # True, если лицо обнаружено
    return len(faces) > 0

# Функция обработки формы страницы регистрации пользователя
# Декоратор, определяющий маршрут для обработки POST-запросов по адресу /api/register
@app.route("/api/register", methods=['POST'])
# Функция обработки запроса регистрации для получения и занесения данных пользователя в БД
def register():
    data = request.json  # получение данных
    # Проверка наличия всех обязательных полей
    required_fields = ['login', 'pass', 'email', 'name']
    # Проверка на наличие данных в запросе     
    if not all(key in data and data[key] for key in required_fields):
        return jsonify({'message': 'Отсутствуют обязательные поля'}), 400
    # Извлечение данных из запроса     
    login = data['login']
    passw = data['pass']
    email = data['email']
    name = data['name']
    # Установка логина для android-приложения по умолчанию
    codeword = "root"
    user_uuid = str(uuid.uuid4())  # создание уникального идентификатора пользователя
    try:
        hashed_password = bcrypt.generate_password_hash(passw).decode('utf-8') # получение хеша пароля
    except ValueError as e:
        return jsonify({'message': str(e)}), 400
    # Формирование запроса для базы данных (проверка существования пользователя)
    cur = mysql.connection.cursor()
    cur.execute("SELECT * FROM users WHERE login = %s OR email = %s", (login, email))
    if cur.fetchone():
        cur.close()
        return jsonify({'message': 'Логин или электронная почта уже существуют'}), 409
    # Формирование запроса для базы данных (внесение данных в БД)
    sql = "INSERT INTO users (login, password, email, name, codeword, uuid) VALUES (%s, %s, %s, %s, %s, %s)"
    val = (login, hashed_password, email, name, codeword, user_uuid)
    cur.execute(sql, val)
    mysql.connection.commit()
    cur.close()
    # Создание access и refresh токенов          
    access_token = create_access_token(identity=login)
    refresh_token = create_refresh_token(identity=login)
    # Запрос к базе данных на установку токена 
    cur = mysql.connection.cursor()
    cur.execute("UPDATE users SET refresh_token = %s WHERE login = %s", (refresh_token, login))
    mysql.connection.commit()
    cur.close()  
    # Создание HTTP-ответа с сообщением о успешной регистрации и установкой куки
    response = make_response(jsonify({'message': 'Регистрация успешна'}), 200)
    response = jsonify({'message': 'Регистрация успешна', 'uuid': user_uuid, 'access_token': access_token, 'refresh_token': refresh_token})
    response.set_cookie('access_token', access_token, httponly=True)
    response.set_cookie('refresh_token', refresh_token, httponly=True)
    response.set_cookie('uuid', user_uuid, httponly=True)    
    return response

# Функция обработки формы страницы авторизации
# Декоратор, определяющий маршрут для обработки POST-запросов по адресу /api/authenticate
@app.route("/api/authenticate", methods=['POST'])
def authenticate():
    session.clear()
    data = request.json
    # Проверка наличия всех обязательных полей
    if not all(key in data for key in ['login', 'pass']):
        return jsonify({'message': 'Отсутствуют обязательные поля'}), 400
    # Извлечение данных из запроса  
    login = data['login']
    passw = data['pass']
    # Формирование запроса для базы данных (проверка существования пользователя)
    cur = mysql.connection.cursor()
    cur.execute("SELECT * FROM users WHERE login = %s", (login,))
    user = cur.fetchone()
    cur.close()
    # Формирование запроса для БД (проверка пароля на верность)
    if user and bcrypt.check_password_hash(user[2], passw):
        user_uuid = user[6]
        access_token = create_access_token(identity=login)
        refresh_token = create_refresh_token(identity=login)
        # Запрос к базе данных на установку токена
        cur = mysql.connection.cursor()
        cur.execute("UPDATE users SET refresh_token = %s WHERE login = %s", (refresh_token, login))
        mysql.connection.commit()
        cur.close()
        # Создание HTTP-ответа с сообщением о успешной регистрации и установкой куки
        response = jsonify({'message': 'Аутентификация успешна', 'uuid': user_uuid, 'access_token': access_token, 'refresh_token': refresh_token})
        response.set_cookie('access_token', access_token, httponly=True)
        response.set_cookie('refresh_token', refresh_token, httponly=True)
        response.set_cookie('uuid', user_uuid, httponly=True)
        return response
    else:
        return jsonify({'message': 'Неверный логин или пароль'}), 401

# Функция обработки формы страницы добавления фото пользователя
# Декоратор, определяющий маршрут для обработки GET и POST-запросов по адресу /api/photo/<uuid>
@app.route("/api/photo/<uuid>", methods=['GET', 'POST'])
@jwt_required() 
def photo(uuid):
    # Получает идентификатор пользователя из JWT токена
    current_user = get_jwt_identity() 
    # Обработка GET-запроса
    if request.method == 'GET':
        # Формирование запроса для БД (для вывода данных)
        cur = mysql.connection.cursor()
        cur.execute("SELECT login, email, name FROM users WHERE uuid = %s", (uuid,))
        user = cur.fetchone()
        cur.close()
        if user:
            login = user[0]
            return jsonify({'login': user[0], 'email': user[1], 'name': user[2]}), 200
        else:
            return jsonify({'message': 'Данные пользователя не найдены'}), 404
    # Обработка POST-запроса
    elif request.method == 'POST':
        if 'file' not in request.files:
            return jsonify({'message': 'Нет файла'}), 405
        # Добавление изображения
        file = request.files['file']
        # Проверка верности добавления изображения
        if file.filename == '':
            return jsonify({'message': 'Не существует выбранного файла'}), 406
        if file and allowed_file(file.filename):
            # Запрос к БД на получение UUID
            cur = mysql.connection.cursor()
            cur.execute("SELECT login FROM users WHERE uuid = %s", (uuid,))
            user = cur.fetchone()
            cur.close()
            if user and user[0] == current_user:
                login = user[0]
                filename = secure_filename(login + '.png')
                if not os.path.exists(app.config['UPLOAD_FOLDER']):
                    os.makedirs(app.config['UPLOAD_FOLDER'])
                image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                file.save(image_path)
                if face(image_path):
                    # Запрос к БД на добавление файла
                    cur = mysql.connection.cursor()
                    sql = "INSERT INTO photo (image_path, name_photo) VALUES (%s, %s)"
                    val = (app.config['UPLOAD_FOLDER'], login)
                    cur.execute(sql, val)
                    mysql.connection.commit()
                    cur.close()
                    return jsonify({'message': 'Фотография загружена успешно', 'filename': filename}), 201
                else:
                    return jsonify({'message': 'На изображении не обнаружено лица'}), 422
            else:
                return jsonify({'message': 'Неверный UUID или доступ запрещен'}), 403
        else:
            return jsonify({'message': 'Недопустимый формат файла, разрешены только файлы в формате PNG'}), 408

# Функция обработки формы страницы регистрации логина для android-приложения
# Декоратор, определяющий маршрут для обработки GET и POST-запросов по адресу /api/codeword/<uuid>   
@app.route("/api/codeword/<uuid>", methods=['GET', 'POST']) 
@jwt_required() 
def codeword(uuid): 
    current_user = get_jwt_identity() 
    # Обработка GET-запроса
    if request.method == 'GET':
        # Формирование запроса для БД (для вывода данных)
        cur = mysql.connection.cursor()
        cur.execute("SELECT login FROM users WHERE uuid = %s", (uuid,))
        user = cur.fetchone()
        cur.close()
        if user:
            loginn = user[0]
            cur = mysql.connection.cursor()
            cur.execute("SELECT u.login, u.email, u.name, p.name_photo FROM users u LEFT JOIN photo p ON u.login = p.name_photo WHERE u.login = %s", (loginn,))
            user = cur.fetchone()
            cur.close()
            # Вывод фотографии
            if user:
                photo_path = user[3] + ".png"
                return jsonify({'login': user[0], 'email': user[1], 'name': user[2], 'photo_path': photo_path}), 200
            else:
                return jsonify({'message': 'Данные пользователя не найдены'}), 404
        else:
            return jsonify({'message': 'Пользователь с указанным uuid не найден'}), 407
    # Обработка POST-запроса
    elif request.method == 'POST':
        data = request.json
        codeword = data.get('codeword')
        # Проверка наличия всех обязательных полей
        if not codeword:
            return jsonify({'message': 'Поле codeword не может быть пустым'}), 415
        # Установка логина для android-приложения
        cur = mysql.connection.cursor()
        cur.execute("UPDATE users SET codeword = %s WHERE uuid = %s", (codeword, uuid))
        mysql.connection.commit()
        cur.close()
        return jsonify({'message': 'Логин для android-приложения обновлен успешно'}), 200

# Функция обработки формы страницы ввода кода
# Декоратор, определяющий маршрут для обработки GET и POST-запросов по адресу /api/autf/<uuid>
@app.route("/api/autf/<uuid>", methods=['GET', 'POST'])
@jwt_required()
def autf(uuid):
    current_user = get_jwt_identity()
    global login
    # Обработка GET-запроса
    if request.method == 'GET':
        # Запрос к БД на получение UUID 
        cur = mysql.connection.cursor()
        cur.execute("SELECT login FROM users WHERE uuid = %s", (uuid,))
        user = cur.fetchone()
        cur.close()
        # Запрос к БД на получение данных пользователя
        if user:
            loginn = user[0]
            cur = mysql.connection.cursor()
            cur.execute("SELECT u.login, u.email, u.name, p.name_photo FROM users u LEFT JOIN photo p ON u.login = p.name_photo WHERE u.login = %s", (loginn,))
            user = cur.fetchone()
            cur.close()
            login = user[0]
            # Получение фотографии
            if user:
                photo_path = user[3] + ".png"
                return jsonify({'login': user[0], 'email': user[1], 'name': user[2], 'photo_path': photo_path}), 200
            else:
                return jsonify({'message': 'Данные пользователя не найдены'}), 404
        else:
            return jsonify({'message': 'Пользователь с указанным uuid не найден'}), 407
    # Обработка POST-запроса
    elif request.method == 'POST':
        data = request.json
        # Получение данных из запроса
        kod = data['code']
        cur = mysql.connection.cursor()
        cur.execute("SELECT * FROM kod WHERE login = %s", (login,))
        hashed = cur.fetchone()
        cur.close()
        if hashed and bcrypt.check_password_hash(hashed[2], kod):
            return jsonify({'success': True, 'message': 'Аутентификация успешна'}), 200
        else:
            return jsonify({'success': False, 'message': 'Неверный код'}), 410

# Функция обработки формы страницы пользователя
# Декоратор, определяющий маршрут для обработки GET-запросов по адресу /api/other/<uuid>      
@app.route("/api/other/<uuid>", methods=['GET']) 
def other(uuid): 
        # Запрос к БД на получение UUID пользователя
        cur = mysql.connection.cursor()
        cur.execute("SELECT login FROM users WHERE uuid = %s", (uuid,))
        user = cur.fetchone()
        cur.close()
        # Запрос к БД на получение данных пользователя        
        cur = mysql.connection.cursor()
        cur.execute("DELETE FROM kod WHERE login = %s", (login,))
        mysql.connection.commit()
        cur.close()     
        # Вывод фотографии пользователя       
        if user:
            loginn = user[0]
            cur = mysql.connection.cursor()
            cur.execute("SELECT u.login, u.email, u.name, p.name_photo FROM users u LEFT JOIN photo p ON u.login = p.name_photo WHERE u.login = %s", (loginn,))
            user = cur.fetchone()
            cur.close()
            if user:
                photo_path = user[3] 
                photo_path = photo_path + ".png"
                return jsonify({'login': user[0], 'email': user[1], 'name': user[2], 'photo_path': photo_path}), 200
            else:
                return jsonify({'message': 'Данные пользователя не найдены'}), 404
        else:
            return jsonify({'message': 'Пользователь с указанным uuid не найден'}), 407
       
# Функция обработки для страницв ввода логина и пароля в android-приложении
# Декоратор, определяющий маршрут для обработки POST-запросов по адресу /andro 
@app.route('/andro', methods=['POST'])
def andro():
    data = request.json
    login = data['username']
    passw = data['password']
    cur = mysql.connection.cursor()
    cur.execute("SELECT * FROM users WHERE codeword = %s", (login,))
    user = cur.fetchone()
    cur.close() 
    if user and bcrypt.check_password_hash(user[2], passw):        
        return jsonify({'success': True, 'message': 'Аутентификация успешна'}), 200
    else:
        return jsonify({'success': False, 'message': 'Неверный логин или пароль'}), 401     

# Функция для отправки кода в android-приложение 
# Декоратор, определяющий маршрут для обработки GET и POST-запросов по адресу /key 
@app.route('/key', methods=['GET', 'POST'])
def key():
    global codekey
    global filename
    global login
    # Обработка POST-запроса
    if request.method == 'POST':
        data = request.json
        login = data['login']
        namephoto = login
        cur = mysql.connection.cursor()
        cur.execute("SELECT image_path, name_photo FROM photo WHERE name_photo = %s", (namephoto,))
        photo = cur.fetchone()
        cur.close()
        if photo:
            file_path = photo[0]
            file_name = photo[1]
            filename = os.path.join(file_path, file_name + ".png")
            print(filename)  
        if photo:
            return jsonify({'success': True, 'message': 'Код обновлен'}), 200
        else:
            return jsonify({'success': False, 'message': 'Фотография не найдена'}), 411
    # Обработка GET-запроса
    elif request.method == 'GET':
        codekey = generate_kod(filename) 
        hashed_code = bcrypt.generate_password_hash(codekey).decode('utf-8')
        key = codekey
        # Удаление кода из БД
        cur = mysql.connection.cursor()
        cur.execute("DELETE FROM kod WHERE login = %s", (login,))
        mysql.connection.commit()
        cur.close()
        # Обновление кода в БД
        cur = mysql.connection.cursor()
        cur.execute("INSERT INTO kod (login, kod) VALUES (%s, %s)", (login, hashed_code))
        mysql.connection.commit()
        cur.close()
        # Отладочный вывод
        print("Ответ перед отправкой:", {'code': key}) 
        return jsonify({'code': key})
        
# Декоратор, определяющий маршрут для обработки запросов по любому адресу      
@app.route("/")
def hello():
    # Отладочный вывод
    return "Hello"

if __name__ == "__main__":
    app.run() # Запуск веб-сервера

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-06-18 20:33:11,105] ERROR in app: Exception on /key [GET]
Traceback (most recent call last):
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask_cors\extension.py", line 178, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\sumarn\AppData\Local\Temp\ipykernel

C:\server\data\htdocs\Diplom\my-app\public\images\sergey.png


[2024-06-18 20:33:48,901] ERROR in app: Exception on /key [GET]
Traceback (most recent call last):
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask_cors\extension.py", line 178, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sumarn\anaconda3\lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\sumarn\AppData\Local\Temp\ipykernel_15088\3160037255.py", line 416, in key
    cur.execute("

Ответ перед отправкой: {'code': '597858'}


...

Ответ перед отправкой: {'code': '710101'}


127.0.0.1 - - [18/Jun/2024 21:48:18] "GET /key HTTP/1.1" 200 -


Ответ перед отправкой: {'code': '681890'}


127.0.0.1 - - [18/Jun/2024 21:48:48] "GET /key HTTP/1.1" 200 -


Ответ перед отправкой: {'code': '553982'}


In [1]:
# !pip install Flask-MySQL
# !pip install --upgrade Flask
# !pip install mysql-connector-python
# !pip install Flask-Bcrypt
# !pip install redis
#  !pip install apscheduler